### 由于时间不足和机器落后，本次比赛只用了LightGBM单模型来训练和预测数据

In [ ]:
import gc

import pandas as pd
import numpy as np

from sklearn.metrics import log_loss

In [ ]:
# 读入Step_6处理后的数据
data = pd.read_csv('../Temp/train_day_7_test_step_6.csv')

In [ ]:
# 为防止前面的数据处理中打乱了data样本的顺序，现重新排序。(因为预测集一定要在训练集后面，不能乱了)
data.sort_values(by=['context_timestamp', 'instance_id'], inplace=True)

In [ ]:
# 删除这几个特征
drop_features = ['context_id', 'context_timestamp', 'day', 'minute', 'second', 'predict_category_property',
                'item_property_list', 'time', 'item_category_list']

In [ ]:
# 线下训练
# 训练集和验证集划分，因为7号总的训练集是107万，预测集50多万，比例大致是2:1的关系，
# 为保持一致，这里划分总的训练集前72万为训练集，后面为验证集
train_set = data[:1077175]
train_set.drop(drop_features, axis=1, inplace=True)

X_train = train_set[:720000]
y_train = X_train['is_trade']
X_valid = train_set[720000:]
y_valid = X_valid['is_trade']

del X_train['is_trade']
del X_valid['is_trade']
gc.collect()

In [ ]:
# 模型训练
import lightgbm as lgb

gbm = lgb.LGBMClassifier(num_leaves=25, max_depth=7, n_estimators=2000, n_jobs=-1, learning_rate=0.05)
gbm.fit(X_train, y_train, eval_set=[(X_valid, y_valid), (X_train, y_train)], eval_names=['eval', 'train'], 
                                    eval_metric='logloss', verbose=True, early_stopping_rounds=20)

In [ ]:
# 线上提交
# 预测结果
test = data[1077175:] # 提取出预测集

# 为保证处理后的预测集样本的排列顺序和原来一致，需要重新导入原始预测集数据，按照原来的instance_id进行排序
test_b = pd.read_csv('../Data/round2_ijcai_18_test_b_20180510.txt', sep='\s+')
test_instance = pd.DataFrame(test_b['instance_id'])
test = pd.merge(test_instance, test, how='left', on='instance_id')
test.drop(drop_features, axis=1, inplace=True)

# 使用全部训练集进行训练
X_train = train_set[:]
y_train = X_train['is_trade']

del X_train['is_trade']
del test['is_trade']
del train_set
gc.collect()

In [ ]:
del data
gc.collect()

In [ ]:
best_iter = gbm.best_iteration_
gbm_test = lgb.LGBMClassifier(num_leaves=25, max_depth=7, n_estimators=best_iter, n_jobs=-1, learning_rate=0.05)
gbm_test.fit(X_train, y_train, eval_metric='logloss')
y_test_gbm = gbm_test.predict_proba(test)[:, 1]

In [ ]:
# 保存预测结果，生成.txt文件提交
test_b['predicted_score'] = y_test_gbm
test_b[['instance_id', 'predicted_score']].to_csv('../Data/submission.txt', index=False, sep=' ')